In [ ]:
from sklearn.datasets import fetch_california_housing
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [ ]:
housing = fetch_california_housing()
housing_df = pd.DataFrame(data= np.c_[housing['data'], housing['target']],
                     columns= housing['feature_names'] + ['target'])

In [ ]:
profile = ProfileReport(housing_df)
profile